# Support Vector Machines for Spam Classifier

In [185]:
def getVocabList(): 
    vocabList = pd.read_csv('vocab.txt', sep = '\t', header = None)
    return vocabList

In [316]:
def processEmail(email_content):
    email_content = email_content.lower()
    
    email_content = re.sub('<[^<>]+>', ' ', email_content)
    email_content = re.sub('[0-9]+','number', email_content)
    email_content = re.sub('(http|https)://[^\s]*', 'httpaddr', email_content)
    email_content = re.sub('[^\s]+@[^\s]+', 'emailaddr', email_content)
    email_content = re.sub('[$]+', 'dollar', email_content)
    email_content = email_content.replace('\n',' ')
    email_content = email_content.translate ({ord(c): " " for c in "!@#$%^&*()[]{};:'',./<>?\|`~-=_+"})
    email_content = re.sub('^ ','', email_content)
    
    ps = PorterStemmer()
    
    vocabList = getVocabList()
    
    words = email_content.split()
    word_indices = []
    for w in words:
        w = re.sub('[^a-zA-Z0-9]', '', w)
        string = ps.stem(w)

        if len(string) < 1:
            continue
        
        try:
            index = vocabList[vocabList.iloc[:,1]==string].iloc[0,0]
            word_indices.append(index)
        except:
            continue
        
    return word_indices

In [221]:
def emailFeatures(word_indices):
    n = 1899
    x = np.zeros((n,1))
    x[word_indices]=1
    return x

# Loading data and training the SVM model

In [333]:
import re
import scipy.io

import numpy as np
import pandas as pd

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn import svm


f = open("emailSample1.txt", "r")
file_content = f.read()
word_indices = processEmail(file_content)
x = emailFeatures(word_indices)

data = scipy.io.loadmat('spamTrain.mat')
X = data['X']
y = data['y'].reshape(-1)

clf = svm.SVC(C=2, kernel='rbf')
clf.fit(X, y)

SVC(C=2)

# Training accuracy

In [335]:
y_predict = clf.predict(X)
accuracy = np.mean(1*(y_predict==y))*100
print('train accuracy: ',accuracy)

train accuracy:  99.675


# Test accuracy

In [334]:
data = scipy.io.loadmat('spamTest.mat')
X_test = data['Xtest']
y_test = data['ytest'].reshape(-1)
y_predict = clf.predict(X_test)
accuracy = np.mean(1*(y_predict==y_test))*100
print('test accuracy: ',accuracy)

test accuracy:  99.1


# Testing a spam example

In [338]:
f = open("spamSample2.txt", "r")
file_content = f.read()
word_indices = processEmail(file_content)
x = emailFeatures(word_indices)

clf.predict(x.reshape(1,-1))

array([1], dtype=uint8)

In [339]:
email_content

'best buy viagra generic online  viagra numbermg x number pills dollarnumber  free pills   reorder discount  top selling number  quality   satisfaction guaranteed   we accept visa  master   e check payments  number  satisfied customers  httpaddr   '